In [3]:
using JuMP, HiGHS

In [9]:
# Datos
oferta = [400, 200, 150]
demanda = [100, 200, 150, 160, 140]
distancia = [
    100 150 200 140 35;
     50  70  60  65 80;
     40  90 100 150 130
]

costo_por_auto = 25 / 18

1.3888888888888888

In [10]:
modelo = Model(HiGHS.Optimizer)

A JuMP Model
├ solver: HiGHS
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

In [11]:
@variable(modelo, x[1:3, 1:5] >= 0) # No negativos

3×5 Matrix{VariableRef}:
 x[1,1]  x[1,2]  x[1,3]  x[1,4]  x[1,5]
 x[2,1]  x[2,2]  x[2,3]  x[2,4]  x[2,5]
 x[3,1]  x[3,2]  x[3,3]  x[3,4]  x[3,5]

In [12]:
@objective(modelo, Min, sum(costo_por_auto * distancia[i, j] * x[i, j] for i in 1:3, j in 1:5))

138.88888888888889 x[1,1] + 208.33333333333331 x[1,2] + 277.77777777777777 x[1,3] + 194.44444444444443 x[1,4] + 48.61111111111111 x[1,5] + 69.44444444444444 x[2,1] + 97.22222222222221 x[2,2] + 83.33333333333333 x[2,3] + 90.27777777777777 x[2,4] + 111.11111111111111 x[2,5] + 55.55555555555556 x[3,1] + 125 x[3,2] + 138.88888888888889 x[3,3] + 208.33333333333331 x[3,4] + 180.55555555555554 x[3,5]

In [13]:
# Restricciones de oferta
for i in 1:3
    @constraint(modelo, sum(x[i, j] for j in 1:5) <= oferta[i])
end

In [14]:
# Restricciones de demanda
for j in 1:5
    @constraint(modelo, sum(x[i, j] for i in 1:3) == demanda[j])
end

In [15]:
optimize!(modelo)

Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [5e+01, 3e+02]
  Bound  [0e+00, 0e+00]
  RHS    [1e+02, 4e+02]
Presolving model
8 rows, 15 cols, 30 nonzeros  0s
8 rows, 15 cols, 30 nonzeros  0s
Presolve : Reductions: rows 8(-0); columns 15(-0); elements 30(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 5(750) 0s
          8     8.7916666667e+04 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 8
Objective value     :  8.7916666667e+04
HiGHS run time      :          0.02


In [16]:
if termination_status(modelo) == MOI.OPTIMAL
    println("Distribución óptima:")
    for i in 1:3, j in 1:5
        println("Enviar ", value(x[i, j]), " autos del Centro $i al Concesionario $j")
    end
    println("Costo total: \$", objective_value(modelo))
else
    println("No se encontró solución óptima.")
end

Distribución óptima:
Enviar 100.0 autos del Centro 1 al Concesionario 1
Enviar 0.0 autos del Centro 1 al Concesionario 2
Enviar 0.0 autos del Centro 1 al Concesionario 3
Enviar 160.0 autos del Centro 1 al Concesionario 4
Enviar 140.0 autos del Centro 1 al Concesionario 5
Enviar 0.0 autos del Centro 2 al Concesionario 1
Enviar 50.0 autos del Centro 2 al Concesionario 2
Enviar 150.0 autos del Centro 2 al Concesionario 3
Enviar 0.0 autos del Centro 2 al Concesionario 4
Enviar 0.0 autos del Centro 2 al Concesionario 5
Enviar -0.0 autos del Centro 3 al Concesionario 1
Enviar 150.0 autos del Centro 3 al Concesionario 2
Enviar 0.0 autos del Centro 3 al Concesionario 3
Enviar 0.0 autos del Centro 3 al Concesionario 4
Enviar 0.0 autos del Centro 3 al Concesionario 5
Costo total: $87916.66666666666
